In [1]:
# use this as as base for CNN model
# https://www.kaggle.com/sunnyguha/basic-cnn-starter

In [2]:
# import images - 1000 to begin with and on a lazy basis
#   need 1000 images that correspond to 1000 y_train
#   images as np.array(1000,size,size,3) and np.array(1000,1)
#   user Keras flow from directory

# CNN Model
#   Layers
#   Compile
#     During compile stage, set the parameter that you want to minimize
#     Weighted column


# View results

# Check
#   In the case of false positives for a class, is it more likely that the result is of a different class or that it is negative in all classes


In [ ]:
# Set global variables
# un-comment as they are used
# batch_size=32
# validation_ratio=0.1
# sample_size=2000
# epochs=3
# img_size=512

# data_path = "../data/data/"

In [ ]:
# Questions for Nick

# where do we input __init__ parameters?
# What does it mean for a class to inherit a sequence

# Input of DataGenerator
# Sequence + (potentially) various __init__ parameters

# Output of DataGenerator
# 